In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import requests
from bs4 import BeautifulSoup

import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

# 구글 무비에서 할인하고 있는 인기차트 추출

- 참고 : 구글 무비와 같이 외국 사이트의 경우 user-agent를 사용하지 않고 추출된 requests 페에지는 언어의 차이가 존재할 수 있다. 따라서 defualt로 항상 사용하자

## BeautifulSoup을 통해 요소 추출의 경우

- 동적 페이지 작동 불가

In [9]:
url = 'https://play.google.com/store/movies/collection/cluster?clp=0g4XChUKD3RvcHNlbGxpbmdfcGFpZBAHGAQ%3D:S:ANO1ljJvXQM&gsr=ChrSDhcKFQoPdG9wc2VsbGluZ19wYWlkEAcYBA%3D%3D:S:ANO1ljK7jAA&hl=ko&gl=US'

header = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
res = requests.get(url, headers = header)
res.raise_for_status()

soup = BeautifulSoup(res.text, 'lxml')

In [18]:
movies = soup.find_all('div', attrs = {'class' : 'ImZGtf mpg5gc'})

for movie in movies:
    title = movie.find('div', attrs = {'class' : 'WsMG1c nnK0zc'}).get_text()
    print(title)

Spider-Man: No Way Home
Sing 2
Venom: Let There Be Carnage
Ghostbusters: Afterlife
Moonfall
Sonic The Hedgehog
Dog
나일 강의 죽음
매트릭스: 리저렉션
The Contractor
Scream
Jackass Forever
No Time To Die
Spider-Man: Far from Home
하우스 오브 구찌
리커리쉬 피자
듄
The Way of the Gun
Marry Me
Boon
Spider-Man: Homecoming
King Richard
Resident Evil: Welcome to Raccoon City
Panama
Belfast
How to Train Your Dragon: The Hidden World
The Outfit (2022)
Alice
Encanto
Letters to Juliet
Clifford The Big Red Dog
씽
The King's Man
Umma
Spider-Man: Into The Spider-Verse
American Underdog
Above Top Secret: The Technology Behind Disclosure
Fifty Shades Freed
Jurassic World
신비한 동물들과 그린델왈드의 범죄
다크 나이트 라이즈
퍼피 구조대 더 무비
Eternals
쥬라기 월드 : 폴른 킹덤
Venom
The Amazing Spider-Man™
Coraline
As They Made Us
신비한 동물사전
Interstellar


## selenium을 통해 동적페이지에 대한 크롤링(스크롤 작업)

- bs4의 경우 동적 페이지 작업이 불가능하다. 이때 구글 무비의 경우 스크롤 다운하게 되면 로딩이 되기 때문에 selenium을 사용해야 한다.

In [34]:
driver = webdriver.Chrome('C:/Users/Park/chromedriver.exe')
url = 'https://play.google.com/store/movies/collection/cluster?clp=0g4XChUKD3RvcHNlbGxpbmdfcGFpZBAHGAQ%3D:S:ANO1ljJvXQM&gsr=ChrSDhcKFQoPdG9wc2VsbGluZ19wYWlkEAcYBA%3D%3D:S:ANO1ljK7jAA&hl=ko&gl=US'

driver.get(url)

### 1. 스크롤 다운

- 지정한 위치로 스크롤 다운
- 예 : (0,1080) -> 1080 아래 위치로 스크롤 다운
> driver.execute_script('window.scollTo(0,y)') : y위치로 스크롤 다운
---------------------------------------
- 화면 가장 아래로 스크롤 다운
> driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')

In [35]:
# 현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script('return document.body.scrollHeight')

while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    time.sleep(3)
    # 가장 아래로 내려진 현재 문서 높이를 가져와서 curr_height에 저장
    curr_height = driver.execute_script('return document.body.scrollHeight')
    if curr_height == prev_height:
        break
        
    prev_height = curr_height
print('스크롤 완료')

스크롤 완료


### 2. bs4를 통해 타이틀 정보 추출

> driver.page_source : 현재 셀레니움으로 동작된 페이지 소스 출력

In [38]:
# 현재 selenium으로 동작된 페이지 소스를 beautifulsoup을 통해 추출

soup = BeautifulSoup(driver.page_source, 'lxml')

for movie in movies:
    title = movie.find('div', attrs = {'class' : 'WsMG1c nnK0zc'}).get_text()
    print(title)

Spider-Man: No Way Home
Sing 2
Venom: Let There Be Carnage
Ghostbusters: Afterlife
Moonfall
Sonic The Hedgehog
Dog
나일 강의 죽음
매트릭스: 리저렉션
The Contractor
Scream
Jackass Forever
No Time To Die
Spider-Man: Far from Home
하우스 오브 구찌
리커리쉬 피자
듄
The Way of the Gun
Marry Me
Boon
Spider-Man: Homecoming
King Richard
Resident Evil: Welcome to Raccoon City
Panama
Belfast
How to Train Your Dragon: The Hidden World
The Outfit (2022)
Alice
Encanto
Letters to Juliet
Clifford The Big Red Dog
씽
The King's Man
Umma
Spider-Man: Into The Spider-Verse
American Underdog
Above Top Secret: The Technology Behind Disclosure
Fifty Shades Freed
Jurassic World
신비한 동물들과 그린델왈드의 범죄
다크 나이트 라이즈
퍼피 구조대 더 무비
Eternals
쥬라기 월드 : 폴른 킹덤
Venom
The Amazing Spider-Man™
Coraline
As They Made Us
신비한 동물사전
Interstellar


#### 모든 페이지 로드 후 실행했더니 태그명이 다른 것이 존재한다.

- 이를 해결하기 위해 attrs={} 내의 class 속성값을 리스트 형태로 작성한다.

In [39]:
movies = soup.find_all('div', attrs = {'class' : ['ImZGtf mpg5gc', 'Vpfmgd']})

for movie in movies:
    title = movie.find('div', attrs = {'class' : 'WsMG1c nnK0zc'}).get_text()
    print(title)

Spider-Man: No Way Home
Spider-Man: No Way Home
Sing 2
Sing 2
Venom: Let There Be Carnage
Venom: Let There Be Carnage
Ghostbusters: Afterlife
Ghostbusters: Afterlife
Moonfall
Moonfall
Sonic The Hedgehog
Sonic The Hedgehog
Dog
Dog
나일 강의 죽음
나일 강의 죽음
매트릭스: 리저렉션
매트릭스: 리저렉션
The Contractor
The Contractor
Scream
Scream
Jackass Forever
Jackass Forever
No Time To Die
No Time To Die
Spider-Man: Far from Home
Spider-Man: Far from Home
하우스 오브 구찌
하우스 오브 구찌
리커리쉬 피자
리커리쉬 피자
듄
듄
The Way of the Gun
The Way of the Gun
Marry Me
Marry Me
Boon
Boon
Spider-Man: Homecoming
Spider-Man: Homecoming
King Richard
King Richard
Resident Evil: Welcome to Raccoon City
Resident Evil: Welcome to Raccoon City
Panama
Panama
Belfast
Belfast
How to Train Your Dragon: The Hidden World
How to Train Your Dragon: The Hidden World
The Outfit (2022)
The Outfit (2022)
Alice
Alice
Encanto
Encanto
Letters to Juliet
Letters to Juliet
Clifford The Big Red Dog
Clifford The Big Red Dog
씽
씽
The King's Man
The King's Man
Umma
Umma
Spider

#### 그런데 중복으로 타이틀이 출력되는 문제가 발생한다.

- 이는 곧 class 속성값이 2개 포함되는 경우가 있기 때문에 이를 찾아서 하나의 속성값만 지정한다.

In [40]:
movies = soup.find_all('div', attrs = {'class' : 'Vpfmgd'})

for movie in movies:
    title = movie.find('div', attrs = {'class' : 'WsMG1c nnK0zc'}).get_text()
    print(title)

Spider-Man: No Way Home
Sing 2
Venom: Let There Be Carnage
Ghostbusters: Afterlife
Moonfall
Sonic The Hedgehog
Dog
나일 강의 죽음
매트릭스: 리저렉션
The Contractor
Scream
Jackass Forever
No Time To Die
Spider-Man: Far from Home
하우스 오브 구찌
리커리쉬 피자
듄
The Way of the Gun
Marry Me
Boon
Spider-Man: Homecoming
King Richard
Resident Evil: Welcome to Raccoon City
Panama
Belfast
How to Train Your Dragon: The Hidden World
The Outfit (2022)
Alice
Encanto
Letters to Juliet
Clifford The Big Red Dog
씽
The King's Man
Umma
Spider-Man: Into The Spider-Verse
American Underdog
Above Top Secret: The Technology Behind Disclosure
Fifty Shades Freed
Jurassic World
신비한 동물들과 그린델왈드의 범죄
다크 나이트 라이즈
퍼피 구조대 더 무비
Eternals
쥬라기 월드 : 폴른 킹덤
Venom
The Amazing Spider-Man™
Coraline
As They Made Us
신비한 동물사전
Interstellar
Pokémon Detective Pikachu
The Wolf of Wall Street
The Prince of Egypt
The Amazing Spider-Man 2
The 355
Spider-Man (2002)
킹스맨: 시크릿 에이전트
Pacific Rim Uprising
Short Circuit
The Cursed
Gold
Tag
The Worst Person in the World
Har

### 2. bs4를 통해 할인된 정보 추출

> driver.page_source : 현재 셀레니움으로 동작된 페이지 소스 출력

In [43]:
for movie in movies:
    title = movie.find('div', attrs = {'class' : 'WsMG1c nnK0zc'}).get_text()
    
    # 할인 전 가격
    original_price = movie.find('span', attrs = {'class' : 'SUZt4c djCuy'})
    # 할인 전 가격이 있다면 
    if original_price:
        # 할인 전 가격 정보 추출
        original_price = original_price.get_text()
    # 없다면 제외
    else:
        continue
        
    # 할인된 가격
    price = movie.find('span', attrs = {'class' : 'VfPpfd ZdBevf i5DZme'}).get_text()
    
    # 링크 정보
    link = str('https://play.google.com') + movie.find('a', attrs = {'class' : 'JC71ub'})['href']
    
    print(f'제목 : {title}')
    print(f'할인 전 금액 : {original_price}')
    print(f'할인 후 금액 : {price}')
    print('링크 : ', link)
    print('-'*100)

제목 : No Time To Die
할인 전 금액 : US$6.99
할인 후 금액 : US$5.99
링크 :  https://play.google.com/store/movies/details/No_Time_To_Die?id=80TMwW2WD3g.P
----------------------------------------------------------------------------------------------------
제목 : The Way of the Gun
할인 전 금액 : US$7.99
할인 후 금액 : US$0.99
링크 :  https://play.google.com/store/movies/details/The_Way_of_the_Gun?id=1SybRysAM7s
----------------------------------------------------------------------------------------------------
제목 : Letters to Juliet
할인 전 금액 : US$9.99
할인 후 금액 : US$0.99
링크 :  https://play.google.com/store/movies/details/Letters_to_Juliet?id=8AjY5M8IGh8
----------------------------------------------------------------------------------------------------
제목 : As They Made Us
할인 전 금액 : US$6.99
할인 후 금액 : US$5.99
링크 :  https://play.google.com/store/movies/details/As_They_Made_Us?id=tvtPea7lDVo.P
----------------------------------------------------------------------------------------------------
제목 : The 355
할인 전 금액 : US$19

In [55]:
title = []
original_price = []
price = []
link = []

for movie in movies:
    title_1 = movie.find('div', attrs = {'class' : 'WsMG1c nnK0zc'}).get_text()
    
    # 할인 전 가격
    original_price_1 = movie.find('span', attrs = {'class' : 'SUZt4c djCuy'})
    # 할인 전 가격이 있다면 
    if original_price_1:
        # 할인 전 가격 정보 추출
        original_price_1 = original_price_1.get_text()
    # 없다면 제외
    else:
        continue
        
    # 할인된 가격
    price_1 = movie.find('span', attrs = {'class' : 'VfPpfd ZdBevf i5DZme'}).get_text()
    
    # 링크 정보
    link_1 = str('https://play.google.com') + movie.find('a', attrs = {'class' : 'JC71ub'})['href']
    
    title.append(title_1)
    original_price.append(original_price_1)
    price.append(price_1)
    link.append(link_1)

result = pd.DataFrame([title,original_price,price,link]).T
result.columns = ['title','original_price','price','link']

result.to_csv('C:/Users/Park/_Python_data/google_movie.csv', encoding = 'utf-8-sig')

# 전체 코드

In [4]:
driver = webdriver.Chrome('C:/Users/Park/chromedriver.exe')
url = 'https://play.google.com/store/movies/collection/cluster?clp=0g4XChUKD3RvcHNlbGxpbmdfcGFpZBAHGAQ%3D:S:ANO1ljJvXQM&gsr=ChrSDhcKFQoPdG9wc2VsbGluZ19wYWlkEAcYBA%3D%3D:S:ANO1ljK7jAA&hl=ko&gl=US'
driver.get(url)

# 현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script('return document.body.scrollHeight')
while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    time.sleep(3)
    # 가장 아래로 내려진 현재 문서 높이를 가져와서 curr_height에 저장
    curr_height = driver.execute_script('return document.body.scrollHeight')
    if curr_height == prev_height:
        break
    prev_height = curr_height

soup = BeautifulSoup(driver.page_source, 'lxml')
movies = soup.find_all('div', attrs = {'class' : 'Vpfmgd'})

title = []
original_price = []
price = []
link = []

for movie in movies:
    title_1 = movie.find('div', attrs = {'class' : 'WsMG1c nnK0zc'}).get_text()    
    # 할인 전 가격
    original_price_1 = movie.find('span', attrs = {'class' : 'SUZt4c djCuy'})
    # 할인 전 가격이 있다면 
    if original_price_1:
        # 할인 전 가격 정보 추출
        original_price_1 = original_price_1.get_text()
    # 없다면 제외
    else:
        continue        
    # 할인된 가격
    price_1 = movie.find('span', attrs = {'class' : 'VfPpfd ZdBevf i5DZme'}).get_text()    
    # 링크 정보
    link_1 = str('https://play.google.com') + movie.find('a', attrs = {'class' : 'JC71ub'})['href']    
    title.append(title_1)
    original_price.append(original_price_1)
    price.append(price_1)
    link.append(link_1)

result = pd.DataFrame([title,original_price,price,link]).T
result.columns = ['title','original_price','price','link']
result.to_csv('C:/Users/Park/_Python_data/google_movie.csv', encoding = 'utf-8-sig')